In [83]:
import olefile, zlib, struct, re

In [84]:
hwp_file_path = r"C:\Users\kooo1\Desktop\11월 계측보고서(우암2구역 주택재개발 정비사업).hwp"

f = olefile.OleFileIO(hwp_file_path)
dirs = f.listdir()

In [85]:
# HWP 파일인지 검증 하는 과정
if ["FileHeader"] not in dirs or ["\x05HwpSummaryInformation"] not in dirs:
    raise Exception("Not Valid HWP")

In [86]:
# 문서 포맷 압축 여부 확인
# 무슨 말인지 정확히는 모르겠음
header = f.openstream("FileHeader")
header_data = header.read()
is_compressed = (header_data[36] & 1) == 1

In [87]:
# Body Section 불러오기
nums = []
for d in dirs:
    if d[0] == "BodyText":
        nums.append(int(d[1][len("Section"):]))

sections = ["BodyText/Section" + str(x) for x in sorted(nums)] 

print(nums)
print(sections)

[0, 1, 2, 3, 4]
['BodyText/Section0', 'BodyText/Section1', 'BodyText/Section2', 'BodyText/Section3', 'BodyText/Section4']


In [89]:
# 전체 text 추출
text = ""
for section in sections:
    bodytext = f.openstream(section)
    data = bodytext.read()
    
    if is_compressed:
        unpacked_data = zlib.decompress(data, -15)
    else:
        unpacked_data = data
        
    section_text = ""
    i = 0
    size = len(unpacked_data)
    
    while i < size:
        header = struct.unpack_from("<I", unpacked_data, i)[0]
        rec_type = header & 0x3ff
        rec_len = (header >> 20) & 0xfff
        
        if rec_type in [67]:
            rec_data = unpacked_data[i+4:i+4+rec_len]
            section_text += rec_data.decode('utf-16')
            section_text += '\n'
            
        i += 4 + rec_len
    
    text += section_text
    text += '\n'

print(text.replace("\r\n", " "))

捤獥    汤捯    湰灧    慤桥    桤灧                                             “우암2구역 주택재개발 정비사업” 발파환경 계측보고서 - 11월 월간보고서 - 2003.  8. 2023. 11. 湰灧    氠瑢     두 산 건 설(주) 무 성 토 건(주) 
汤捯    捤獥    湰灧     제    출    문   氠瑢     두 산 건 설(주) 무 성 토 건(주) 현장소장 귀하  桤灧     귀사에서 의뢰하신 “우암2구역 주택 재개발 정비사업” 중 2023년 11월 발파환경 계측분석이 완료되었기에 그 내용을 수록한 보고서를 제출합니다. 2023. 11.   氠瑢     漠杳     거 명 E & C㈜ 대표이사 박  종  호 (화약류관리기술사/공학박사(수료)) 桤灧     참 여 기 술 진 氠瑢    漠杳     구    분 분    야 직   위 성    명 자격사항 참여기술자 사업 총괄 대표이사 박 종 호 화약류관리기술사 공학박사(수료) 총괄 책임자 기술이사 김 영 민 특급기술자 진동/소음 차  장 강 유 정 소음진동기사 진동/소음 과 장 최 지 혜 초급기술자 책임 기술자 서       명  화약류관리기술사 박 종 호   
汤捯    捤獥    < 목  차 >潴景    慤桥    桤灧                                                                      潣╴    제 1 장 사업 개요	阼 ȃ   	1 1.1. 개요	Ꝙ ȃ   	1 1.2. 과업 목적	鴬 ȃ   	1 1.3. 굴착지역 인접 보안 물건 현황	矬 ȃ   	2   1.3.2.   보안물건 특성	覰 ȃ   	3   1.3.3.   총포도검화약류 등 안전에 관한 법률에 따른 보안물건 분류	㢔 ȃ   	6   1.3.4.   국토의 계획 및 이용에 관한 법률에 따른 지역지구에 따른 분류	ㄠ ȃ   	7 제 2 장 지형 

In [68]:
text = list(filter(lambda x: x, text))

print(text)

['\x02捤獥\x00\x00\x00\x00\x02\x02汤捯\x00\x00\x00\x00\x02\x15湰灧\x00\x00\x00\x00\x15\x10慤桥\x00\x00\x00\x00\x10\x15桤灧\x00\x00\x00\x00\x15', '“신안산선', '복선전철', '건설공사', '제2공구”', '월간', '발파', '계측보고서', '-', '23년', '2월', '월간보고서', '-', '2003.', '8.', '2023.', '02.', '\x0b氠瑢\x00\x00\x00\x00\x0b', '\x0b漠杳\x00\x00\x00\x00\x0b', '포스코건설', '\x0b漠杳\x00\x00\x00\x00\x0b', '금풍건설이엔씨', '\n\x02汤捯\x00\x00\x00\x00\x02\x02捤獥\x00\x00\x00\x00\x02\x15湰灧\x00\x00\x00\x00\x15', '\x1f제', '출', '문', '\x0b氠瑢\x00\x00\x00\x00\x0b', '금풍건설이엔씨(주)', '현장소장', '귀하', '\x15桤灧\x00\x00\x00\x00\x15', '귀', '사에서', '의뢰하신', '“신안산선', '복선전철', '건설공사', '제2공구', '현장”', '중', '23년', '02월', '발파환경', '계측', '자료의', '분석', '및', '검토를', '완료되었기에', '그', '내용을', '수록한', '보고서를', '제출합니다.', '2023.', '02', '\x0b氠瑢\x00\x00\x00\x00\x0b', '\x0b漠杳\x00\x00\x00\x00\x0b', '거', '명', 'E', '&', 'C㈜', '대표이사', '박', '종', '호', '(화약류관리기술사/공학박사(수료))', '\x15桤灧\x00\x00\x00\x00\x15', '참', '여', '기', '술', '진', '\x0b氠瑢\x00\x00\x00\x00\x0b\x0b漠杳\x00\x00\x00\x00\x0b', '구', '분', '분', '야', '직'

In [74]:
find_str = "계측한"
find_key = None

for i in text:
    

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals